# T1040 - Network Sniffing
Adversaries may sniff network traffic to capture information about an environment, including authentication material passed over the network. Network sniffing refers to using the network interface on a system to monitor or capture information sent over a wired or wireless connection. An adversary may place a network interface into promiscuous mode to passively access data in transit over the network, or use span ports to capture a larger amount of data.

Data captured via this technique may include user credentials, especially those sent over an insecure, unencrypted protocol. Techniques for name service resolution poisoning, such as [LLMNR/NBT-NS Poisoning and SMB Relay](https://attack.mitre.org/techniques/T1557/001), can also be used to capture credentials to websites, proxies, and internal systems by redirecting traffic to an adversary.

Network sniffing may also reveal configuration details, such as running services, version numbers, and other network characteristics (e.g. IP addresses, hostnames, VLAN IDs) necessary for subsequent Lateral Movement and/or Defense Evasion activities.

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/haresudhan/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - Packet Capture Linux
Perform a PCAP. Wireshark will be required for tshark. TCPdump may already be installed.

Upon successful execution, tshark or tcpdump will execute and capture 5 packets on interface ens33.

**Supported Platforms:** linux
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Check if at least one of the tools are installed on the machine.

##### Check Prereq Commands:
```bash
if [ ! -x "$(command -v tcpdump)" ] && [ ! -x "$(command -v tshark)" ]; then exit 1; else exit 0; fi;

```
##### Get Prereq Commands:
```bash
echo "Install tcpdump and/or tshark for the test to run."; exit 1;

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
tcpdump -c 5 -nnni ens33
tshark -c 5 -i ens33
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 1

### Atomic Test #2 - Packet Capture macOS
Perform a PCAP on macOS. This will require Wireshark/tshark to be installed. TCPdump may already be installed.

Upon successful execution, tshark or tcpdump will execute and capture 5 packets on interface en0A.

**Supported Platforms:** macos
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Check if at least one of the tools are installed on the machine.

##### Check Prereq Commands:
```bash
if [ ! -x "$(command -v tcpdump)" ] && [ ! -x "$(command -v tshark)" ]; then exit 1; else exit 0; fi;

```
##### Get Prereq Commands:
```bash
echo "Install tcpdump and/or tshark for the test to run."; exit 1;

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo tcpdump -c 5 -nnni en0A    
if [ -x "$(command -v tshark)" ]; then sudo tshark -c 5 -i en0A; fi;
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 2

### Atomic Test #3 - Packet Capture Windows Command Prompt
Perform a packet capture using the windows command prompt. This will require a host that has Wireshark/Tshark
installed.

Upon successful execution, tshark will execute and capture 5 packets on interface "Ethernet".

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: tshark must be installed and in the default path of "c:\Program Files\Wireshark\Tshark.exe".

##### Check Prereq Commands:
```powershell
if (test-path "c:\program files\wireshark\tshark.exe") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest -OutFile $env:temp\wireshark_installer.exe https://2.na.dl.wireshark.org/win64/Wireshark-win64-3.2.6.exe
Start-Process $env:temp\wireshark_installer.exe /S

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"c:\Program Files\Wireshark\tshark.exe" -i Ethernet -c 5
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 3

### Atomic Test #4 - Windows Internal Packet Capture
Uses the built-in Windows packet capture
After execution you should find a file named trace.etl and trace.cab in the temp directory
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
netsh trace start capture=yes tracefile=%temp%\trace.etl maxsize=10```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 4

## Detection
Detecting the events leading up to sniffing network traffic may be the best method of detection. From the host level, an adversary would likely need to perform a man-in-the-middle attack against other devices on a wired network in order to capture traffic that was not to or from the current compromised system. This change in the flow of information is detectable at the enclave network level. Monitor for ARP spoofing and gratuitous ARP broadcasts. Detecting compromised network devices is a bit more challenging. Auditing administrator logins, configuration changes, and device images is required to detect malicious changes.

# Shield Active Defense
### Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
#### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
#### Use Case
By changing the output of network sniffing utilities normally found on a system, you can prevent adversaries from seeing particular content or making use of the results at all.
#### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.